# Water Region Segmentation

This report aims to provide an automation solution to the problem of identifying water regions in satellite images.

Satellite images from programs such as Sentinel-2 can provide time series images of specific resolutions which can then be used to analyze changes in the area of water region such as lakes over time.

The class `WaterRegion` is provided and examples are shown here that makes use of deep learning network trainined over satellite images to segment the region of interest.

Please note that throughout this document, pre-requisite for this facility is marked with the annotation: [*]

## WaterRegion

The main service provided by the `WaterRegion` class is to load the trained deep neural network and makes predictions along with some utility methods to make predictions better and measure the quality of the prediction.

### Loading Satellite Images

The interface or the `WaterRegion` class expects the images to be in the form of numpy array [*]. There are several facilities available outside of this module, however, method `convertFromJP2` would return a numpy array from a JP2 file. The example below shows satellite image of Lake Geneva, 2019 on the left and 2020 on the right.

```
input_image_1 = water_region.convertFromJP2('DemoData/T31TGM_20190629T103031_TCI_10m.jp2')

input_image_2 = water_region.convertFromJP2('DemoData/Geneva_T31TGM_20200623T103031_TCI_10m.jp2')

```

![Lake Geneva](Resources/LakeGeneva.png)

The images also needs to be of size 256 x 256 [*]. The conversion is left outside of the module to give the user, the flexibility to control and maintain spatial resolution.

### Water Region Segmentation

The ouptut or the prediction of the network is an image mask localizing the water region within the input image. The output of the method `identifyRegion` for the above two images are as follows:

```
preds = water_region.identifyRegion(np.array([input_image_1, input_image_2]))
```

![Lake Geneva Predictions Without Change](Resources/LakeGeneva_pred_unfiltered.png)

These masks do have some noise from the snow and part of another lake from the 2019 image is segmented but not in 2020. I've created some simple utility method that we can try on these images.

Another example is from Copais lake. The image on the left is from 2016 and the one on the right is from 2019.

![Copais Lake](Resources/CopaisLake.png)

The corresponding masks are as follows:

![Copais Mask Predictions](Resources/CopaisLake_pred__unfiltered.png)

There is a bit more noise on the 2019 image and you may also notice that the lakes in the original satellite images from the two years do look a little different.

### Comparing Predicted Masks

#### Intersection Over Union

We can measure how much the masks are similar to each other using a concept called `Intersection over Union`. The `WaterRegion` class provides a method called `iou` that calculates the intersection area of the two image masks and divide it with the union of the two masks.

Let's say we have these two images. First one is red, and the second one blue.

![Block Images](Resources/iou_seperate.png)

You can see above that the middle area is common in both and both have some extra bits set, upper left for the first image and bottom right for the second image. The composite image for the above two are shown below:

![Composite](Resources/iou_composite.png)

An `IoU` would give us an indication of how similar the two masks are. For the above images the iou returned is: **`0.8`**

A perfect overlap would result in an IoU of **`1`**.

#### Net Change

Another method provided in the `WaterRegion` class is `netChange`. It `subtracts` the first image from the second image and obtains the net change between the predicted mask. However, a caveat of it being the net change is that if the other lake gained as much as what it had lost in some other localized area, the net change will neutralize that. The netchange for the two left images is **`2`** as you can see in the right most composite image, the two red pixels on the top left corner are missing in the second image.

![Block Images](Resources/composite_netchange.png)

### Utility Methods

#### Defrost

I've created some simple utility method to help process either the image or the mask.

The `defrost` method can be used to remove snow from the Genova Lake images before making prediction on it.

Let's see the original image with composition side-by-side again.

![Lake Geneva Composite Mask](Resources/LakeGeneva_pred_unfiltered_composite.png)

The `IoU` for this is **`0.719`**. This indicates it's close but clearly its lower because of the extra noise that we are seeing due to snow.

Running the `defrost` function makes the following changes to the original images.

![Lake Genova Defrosted](Resources/LakeGeneva_defrosted.png)

The corresponding masks are below with the `IoU` value of **`0.9096`**.

![Lake Geneva Predicted Masks](Resources/LakeGeneva_masks_defrosted.png)

#### Denoise

The `denoise` method averages the area surrounding each pixels, calculated using strides, and clips it's result to 0 or 1 based on the threshold. Let's look at the Copais lakes again.

![Copais Lake](Resources/CopaisLake.png)

The original masks are as follows with **`IoU = 0.64`** and **`net change = -4527`**. 

![Original Masks](Resources/CopaisLake_pred_comosite_unfiltered.png)

After running denoise, following masks are produced with **`IoU = 0.6933`** and **`net change = -2972`**.

![Copais Lake Masks After Denoising](Resources/CopaisLake_pred_comosite_denoised.png)

You can see in the cleaned up mask that the water identified by another lake seems to have increased a little and the 2016 image of the Copais lake shows perhaps a dried up region at the tail of the lake which shows water more clearly in the 2019 image.

# Benchmark

Following results were produced on `Intel(R) Core(TM) i7-4940MX CPU @ 3.10GHz`. It took `6.88` seconds to predict 100 masks, `84.35` seconds for 1000 masks, and `753.79` to predict 10,000 masks.

![timeit results](Resources/timeit_graph.png)

# Conclusion

Automation pipeline can be created that includes downloading satellite images periodically from programs such as Copernicus, the images can then be resized as required by the neural network to make predictions. A time series comparison of the lakes with lower IoU and higher net change values could predicate a significant change. 

Because the masks produced may have some noise, a threshold should be used to identify if indeed a significant change has occurred.